In [1]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from geofree.util import get_local_path, OmegaConf
from geofree.main import instantiate_from_config
import torchvision.transforms as transforms
import torch

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import torchvision.models as models
import copy

model="re_impl_nodepth"
prefix = model[:2]
assert prefix in ["re", "ac"], "not implemented"

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"



c:\Users\USER\anaconda3\envs\geofree\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

config_path = get_local_path(model+"_config")
config = OmegaConf.load(config_path)
config.model.params.first_stage_config.params["ckpt_path"] = get_local_path(f"{prefix}_first_stage")

config.model.params.depth_stage_config.params["ckpt_path"] = get_local_path(f"{prefix}_depth_stage")

vqgan_config = config.model.params.first_stage_config
print(config) 

{'model': {'base_learning_rate': 0.0625, 'target': 'geofree.models.transformers.geogpt.GeoTransformer', 'params': {'use_depth': False, 'plot_cond_stage': True, 'monitor': 'val/loss', 'use_scheduler': True, 'scheduler_config': {'target': 'geofree.lr_scheduler.LambdaWarmUpCosineScheduler', 'params': {'verbosity_interval': 0, 'warm_up_steps': 5000, 'max_decay_steps': 500001, 'lr_start': 2.5e-06, 'lr_max': 0.00015, 'lr_min': 1e-08}}, 'transformer_config': {'target': 'geofree.modules.transformer.mingpt.GPT', 'params': {'vocab_size': 16384, 'block_size': 627, 'n_unmasked': 329, 'n_layer': 32, 'n_head': 16, 'n_embd': 1024}}, 'first_stage_key': {'x': 'dst_img'}, 'cond_stage_key': {'c': 'src_img'}, 'emb_stage_key': {'points': 'src_points', 'R': 'R_rel', 't': 't_rel', 'K': 'K', 'K_inv': 'K_inv'}, 'first_stage_config': {'target': 'geofree.models.vqgan.VQModel', 'params': {'ckpt_path': 'C:\\Users\\USER/.cache\\geofree/re_first_stage/last.ckpt', 'embed_dim': 256, 'n_embed': 16384, 'ddconfig': {'dou

In [3]:
model = instantiate_from_config(vqgan_config)

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
Restored from C:\Users\USER/.cache\geofree/re_first_stage/last.ckpt


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
print(f"Using device: {device}")


Using device: cuda


In [6]:
def get_indeces_of_images(images_path,saved_codes_path):
    #Scene name as the name of the folder
    scene_name = images_path.split("/")[-3]
    #Check if the scene has been processed before
    if os.path.exists(saved_codes_path + scene_name + ".npy"):
        print(f"Codes for {scene_name} already exist")
        return None
    images = os.listdir(images_path)
    codes = []
    print("Number of images: ", len(images), " in ", scene_name)
    with torch.no_grad():
        for i in range(len(images)):
            img = Image.open(images_path + images[i])
            # Resize to 368x208
            img = img.resize((368, 208))
            img = transforms.ToTensor()(img).unsqueeze(0).to(device)
            quant, _, info = model.encode(img)
            indices = info[2].view(quant.shape[0], -1)
            codes.append(indices[0].cpu().numpy())

    codes = np.array(codes)
    print(f"Codes shape for {scene_name}: ", codes.shape)

    #Save the codes
    np.save(saved_codes_path + scene_name + ".npy", codes)
    print(f"Codes for {scene_name} saved")


In [9]:
dataset_path = "Realstate10K/RealEstate10K_Downloader/RealEstate10K/test/"
saved_codes_path = "RealEstate10K_Downloader/RealEstate10K/saved_codes/"
model.to(device)
for scene in os.listdir(dataset_path):
    if scene == "txt_files" or scene == "data.zip":
        continue
    get_indeces_of_images(dataset_path + scene + "/frames/", saved_codes_path)


Codes for 0000cc6d8b108390 already exist
Codes for 000c3ab189999a83 already exist
Codes for 000d2139ad05dc85 already exist
Codes for 000db54a47bd43fe already exist
Codes for 000e285b03f3fddf already exist
Codes for 00a50bfbce75d465 already exist
Codes for 00a5a09a0c68b59b already exist
Codes for 00a5cfccd6508e96 already exist
Codes for 00a7a9c0ea61670d already exist
Codes for 00a861d699fb7797 already exist
Codes for 00a87dd567cdc292 already exist
Codes for 00a9f110ad222aa4 already exist
Codes for 0a0b1ce8a70f8c0c already exist
Codes for b053f2ffb587f61a already exist
Codes for b054fb5ce2f8e1f2 already exist
Codes for b53a8e8d3caa0f66 already exist
Codes for b53b27fdad29cf8d already exist
Codes for b53bba5522b4b8f1 already exist
Codes for b53c5559170c4f78 already exist
Codes for b53cda0b95d897a8 already exist
Codes for b53cea58c1e95448 already exist
Codes for b53dacad44c35ce1 already exist
Codes for b53e3ec3e9b46c8f already exist
Number of images:  212  in  fe002afe708218f0
Codes shape 

: 